# Imports

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.auto import tqdm
from scipy import sparse
import torch
import wandb

import warnings
warnings.filterwarnings('ignore')

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hari31416. Use `wandb login --relogin` to force relogin


True

# Data

## Load data

In [8]:
wandb.init(project='Book_Reccomondation', name='MLP_Model', notes='Uses MLP Model for creating embedding vectors and then passes it to MLP layer for prediction', tags=['MLP', 'Modeling', "Pytorch"])

In [9]:
DATA_DIR = os.path.join("..", "data", "final_dataset")

In [10]:
interactions = pd.read_parquet(os.path.join(DATA_DIR, "ratings_final.parquet"))
users = pd.read_parquet(os.path.join(DATA_DIR, "users_final.parquet"))
books = pd.read_parquet(os.path.join(DATA_DIR, "books_final.parquet"))
user_id_map_df = pd.read_csv(os.path.join(DATA_DIR, "user_id_map.csv"))
book_id_map_df = pd.read_csv(os.path.join(DATA_DIR, "book_id_map.csv"))

print(f"Number of ratings: {len(interactions)}")
print(f"Number of unique users: {interactions['user_id'].nunique()}")
print(f"Number of books: {interactions['book_id'].nunique()}")
print(f"Shape of interactions: {interactions.shape}")
print(f"Shape of books: {books.shape}")
print(f"Shape of users: {users.shape}")

Number of ratings: 104756
Number of unique users: 31940
Number of books: 22020
Shape of interactions: (104756, 3)
Shape of books: (22058, 13)
Shape of users: (31940, 35)


In [11]:
m = len(users)
n = len(books)
print(f"There are {m} unique users and {n} unique books in this data set")

There are 31940 unique users and 22058 unique books in this data set


In [12]:
users.set_index('user_id', inplace=True)
books.set_index('book_id', inplace=True)
books.sort_index(inplace=True)

In [13]:
m_f = users.shape[1]
n_f = books.shape[1]
print(f"There are {m_f} user features and {n_f} book features")

There are 34 user features and 12 book features


## Split

In [14]:
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

train_interactions, test_interactions = split_dataframe(interactions)

# Theory

Using neural networks instead of matrix factorization (MF) for recommendation systems has a number of advantages. See the notes section for detail. Here, we will be providing the model architecture that will be used for the recommendation system.

The model will consist of two steps:

## The Embedding Layer

This layer will take the user and item IDs in one-hot encoded form along with any other user and item feature and will pass it through a fully connected layer. The output of this layer will be the latent representation of the user and item. Let use denote $\mathbf{u}$ as the user and $\mathbf{v}$ as the item. Their dimensions will be $m+m_{uf}$ and $n+n_{if}$ respectively where $m$ and $n$ are the number of users and items and $m_{uf}$ and $n_{if}$ are the number of user and item features respectively. The output of the embedding layer will be $\mathbf{u} \in \mathbb{R}^d$ and $\mathbf{v} \in \mathbb{R}^d$ where $d$ is the dimension of the latent space.

We will have two different layers, one for users and the other of items. This is required because the number of users and items are different and we want to learn different embeddings for them. The user embedding layer will have $m+m_{uf}$ neurons and the item embedding layer will have $n+n_{if}$ neurons.

## CF Layers

CF layers, or collaborative filtering layers are made up of one or more layers of fully connected layers. The input to these layers will be the concatenation of the user and item latent representations. The output of the CF layers will be the predicted rating.

![](images/dl_01.png)

## Loss Function

The loss function can either be MSE or cross entropy. We will be experimenting with both.

## Data

We will be using all the positive data and a random sample of the negative data. The ratio of positive to negative data will be decided by a parameter.

# Implementation

## Dataset

The model will be trained using the one-hot representation of the user and item ids along with the user and item features. The output will be the rating. So, the input will be two vectors of length $M = m+m_{uf}$ and $N = n+n_{if}$ and the output will be a scalar. This means that we will be using pointwise approach.

### Negative Sampling

We will be using negative sampling to train the model. This is important, as we want our model to learn that the rating of a user-item pair is zero if the user has not rated the item. We will define a variable `negative_samples_ratio` that can be used to control the ratio of positive to negative samples. The negative samples will be randomly sampled from the negative data. We can set `negative_samples_ratio` to 0.3-0.5. This means that for every positive sample, we will be using 0.3-0.5 negative samples.

### Dataset Class

Here is a class that creates a dataset using the one-hot representation of the user and item ids along with the user and item features.

In [101]:
class BookDataset(torch.utils.data.Dataset):
    """Dataset class for the model."""""
    def __init__(self, users, books, interactions, config):
        """Initializes the BookDataset.
        
        Parameters
        ----------
        users: pandas.DataFrame
            DataFrame containing user features.
        books: pandas.DataFrame
            DataFrame containing book features.
        interactions: pandas.DataFrame
            DataFrame containing user-book interactions.
        negative_samples_ratio: float
            Ratio of negative samples to positive samples. Must be between 0 and 1.
        features: bool
            If True, the model will use features. If False, the model will not use features.
        """
        self.negative_samples_ratio = config.negative_samples_ratio
        self.features = config.features
        self.normalize = config.normalize
        self.users = users
        self.books = books
        self.interactions = interactions.copy()
        self.m = len(self.users)
        self.n = len(self.books)
        self.m_f = self.users.shape[1]
        self.n_f = self.books.shape[1]
        if self.normalize:
            self.normalize_ratings()
        if self.negative_samples_ratio < 0 or self.negative_samples_ratio > 1:
            raise ValueError("negative_samples_ratio must be between 0 and 1.")
        self.negative_samples_ratio = self.negative_samples_ratio

    def normalize_ratings(self):
        """Normalizes the ratings by dividing by 10."""
        # mean = self.interactions["provided_rating"].mean()
        # std = self.interactions["provided_rating"].std()
        # self.mean = mean
        # self.std = std
        # self.interactions["provided_rating"] = (self.interactions["provided_rating"] - self.interactions["provided_rating"].mean())/self.interactions["provided_rating"].std()
        self.interactions["provided_rating"] = (self.interactions["provided_rating"]/10).round(2)
    
    def decode_rating(self, rating):
        """Decodes a rating."""
        # return rating*self.std + self.mean
        if not self.normalize:
            return rating
        return round(rating*10, 0)
        
    def get_user_features(self, user_id):
        # Assumes that the user_id column is the index in the users DataFrame.
        user = self.users.iloc[user_id].values
        return user.reshape((self.m_f,))

    def get_book_features(self, book_id):
        # Assumes that the book_id column is the index in the books DataFrame.
        book = self.books.iloc[book_id].values
        return book.reshape((self.n_f,))

    def __len__(self):
        # tried this but this will give error as the interaction dataframe will become out of index
        # num_times = 1+self.negative_samples_ratio
        num_times = 1
        return int(len(self.interactions)*num_times)
    
    def get_positive_sample(self, idx):
        """Gets a positive sample from the interactions dataframe."""
        row = self.interactions.iloc[idx]
        user_id = row["user_id"]
        book_id = row["book_id"]
        rating = row["provided_rating"]
        user_id = np.array([user_id])
        book_id = np.array([book_id])
        if self.features:
            user_features = self.get_user_features(user_id)
            # user_id = user_id.reshape((1, 1))
            book_features = self.get_book_features(book_id)
            # book_id = book_id.reshape((1, 1))
            user_input = np.concatenate([user_id, user_features], axis = -1)
            book_input = np.concatenate([book_id, book_features], axis = -1)
        else:
            user_input = user_id
            book_input = book_id

        # make sure the length of the input is correct
        if self.features:
            assert len(user_input) == 1 + self.m_f
            assert len(book_input) == 1 + self.n_f
        else:
            assert len(user_input) == 1
            assert len(book_input) == 1
        return user_input, book_input, rating
    
    def get_negative_sample(self, idx):
        """Gets a negative sample from the interactions dataframe."""""
        row = self.interactions.iloc[idx]
        user_id = row["user_id"]
        negative_book_id = np.random.choice(self.books.index.values)
        while negative_book_id in self.interactions[self.interactions["user_id"] == user_id]["book_id"].values:
            negative_book_id = np.random.choice(self.books.index.values)

        user_id = np.array([user_id])
        book_id = np.array([negative_book_id])
        if self.features:
            user_features = self.get_user_features(user_id)
            # user_id = user_id.reshape((1, 1))
            book_features = self.get_book_features(book_id)
            # book_id = book_id.reshape((1, 1))
            # add user_id at the beginning of the user_features array
            user_input = np.concatenate([user_id, user_features], axis = -1)
            book_input = np.concatenate([book_id, book_features], axis = -1)
        else:
            user_input = user_id
            book_input = book_id

        # make sure the length of the input is correct
        if self.features:
            assert len(user_input) == 1 + self.m_f
            assert len(book_input) == 1 + self.n_f
        else:
            assert len(user_input) == 1
            assert len(book_input) == 1
            
        rating = 0 # negative sample
        return user_input, book_input, rating
    
    def get_one_sample(self, idx):
        """Gets one sample from the dataset. Uses negative sampling with probability `negative_samples_ratio`."""
        if np.random.random() < self.negative_samples_ratio:
            return self.get_negative_sample(idx)
        else:
            return self.get_positive_sample(idx)

    def __getitem__(self, idx):
        """Gets one sample from the dataset."""
        # A workaround to make interaction dataframe circular when using negative sampling
        # with `num_times` > 1. Leaving it as this should not be necessary here.
        # if idx >= len(self.interactions):
        #     idx = idx%len(self.interactions)
        user_input, book_input, rating = self.get_one_sample(idx)
        if self.features:
            dtype = torch.float32
        else:
            dtype = torch.long
        user_input = torch.tensor(user_input, dtype=dtype)
        book_input = torch.tensor(book_input, dtype=dtype)
        targets = torch.tensor(rating, dtype=torch.float32)
        return user_input, book_input, targets

In [67]:
config = wandb.config
config.negative_samples_ratio = 0.5
config.features = False
config.normalize = True
book_dataset = BookDataset(users, books, interactions, config)
book_dataset_batched = torch.utils.data.DataLoader(book_dataset, batch_size=32, shuffle=True)
user_input, book_input, targets = next(iter(book_dataset_batched))
(targets==0).sum()

tensor(13)

> When we use `negative_samples_ratio = 0.5` we usually get 17-20 negative samples for each positive sample. This suggests that we can get away with a lower ratio.

Let use see if the negative sampling is working as expected.

In [55]:
negative_samples = np.where(targets.numpy()==0)
for idx in negative_samples[0][:5]:
    rating = targets[idx].item()
    print(f"Rating: {rating}")
    user_id = user_input[idx].item()
    book_id = book_input[idx].item()
    display(interactions[(interactions["user_id"] == user_id) & (interactions["book_id"] == book_id)])

Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


In [56]:
positive_samples = np.where(targets.numpy()!=0)
for idx in positive_samples[0][:5]:
    rating = targets[idx].item()
    print(f"Rating: {rating}")
    user_id = user_input[idx].item()
    book_id = book_input[idx].item()
    display(book_dataset.interactions[(book_dataset.interactions["user_id"] == user_id) & (book_dataset.interactions["book_id"] == book_id)])

Rating: 0.800000011920929


,user_id,book_id,provided_rating
5503,1273,3700,0.8


Rating: 1.0


,user_id,book_id,provided_rating
103454,31449,2500,1.0


Rating: 0.8999999761581421


,user_id,book_id,provided_rating
43478,12741,8018,0.9


Rating: 0.8999999761581421


,user_id,book_id,provided_rating
38004,11323,98,0.9


Rating: 0.800000011920929


,user_id,book_id,provided_rating
65347,19577,6778,0.8


In [68]:
config = wandb.config
config.negative_samples_ratio = 0.5
config.features = True
config.normalize = True
book_dataset = BookDataset(users, books, interactions, config)
book_dataset_batched = torch.utils.data.DataLoader(book_dataset, batch_size=32, shuffle=True)
user_input, book_input, targets = next(iter(book_dataset_batched))
(targets==0).sum()

tensor(20)

In [61]:
negative_samples = np.where(targets.numpy()==0)
for idx in negative_samples[0][:5]:
    rating = targets[idx].item()
    print(f"Rating: {rating}")
    user_id = user_input[idx][0].item()
    book_id = book_input[idx][0].item()
    display(interactions[(interactions["user_id"] == user_id) & (interactions["book_id"] == book_id)])

Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


Rating: 0.0


,user_id,book_id,provided_rating


In [62]:
positive_samples = np.where(targets.numpy()!=0)
for idx in positive_samples[0][:5]:
    rating = targets[idx].item()
    print(f"Rating: {rating}")
    user_id = user_input[idx][0].item()
    book_id = book_input[idx][0].item()
    display(book_dataset.interactions[(book_dataset.interactions["user_id"] == user_id) & (book_dataset.interactions["book_id"] == book_id)])

Rating: 0.800000011920929


,user_id,book_id,provided_rating
32532,9799,1540,0.8


Rating: 0.6000000238418579


,user_id,book_id,provided_rating
58561,17510,3006,0.6


Rating: 0.699999988079071


,user_id,book_id,provided_rating
22,17,21,0.7


Rating: 0.699999988079071


,user_id,book_id,provided_rating
52604,15617,4716,0.7


Rating: 0.5


,user_id,book_id,provided_rating
32278,9690,5243,0.5


It is working as expected.

We will create the final train and test datasets.

In [685]:
negative_samples_ratio = 0.4
batch_size = 32
train_dataset = BookDataset(users, books, train_interactions, negative_samples_ratio=negative_samples_ratio)
test_dataset = BookDataset(users, books, test_interactions, negative_samples_ratio=negative_samples_ratio)

train_df = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_df = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

Now, we will create the model.

## Model MLP

We will use $d = 40$ for the embedding layer. We will use 2 layers for the CF layers. We can treat these as hyperparameters and tune them later.

In [113]:
class MLPModel(torch.nn.Module):
    """Model class for the BookNet model."""
    def __init__(self, config):
        """Initializes the BookNet model.
        
        Parameters
        ----------
        m: int
            Number of users.
        n: int
            Number of books.
        m_f: int
            Number of user features.
        n_f: int
            Number of book features.
        embedding_dim: int
            Hidden dimension for the hidden layer.
        cf_layer_neurons: list
            List of integers specifying the number of neurons in each layer of the collaborative filtering part of the model.
        """
        super(MLPModel, self).__init__()
        m = config.m
        n = config.n
        m_f = config.m_f
        n_f = config.n_f
        embedding_dim = config.embedding_dim
        cf_layer_neurons = config.cf_layer_neurons
        self.m = m
        self.n = n
        self.m_f = m_f
        self.n_f = n_f
        self.embedding_dim = embedding_dim
        self.cf_layer_neurons = cf_layer_neurons

        self.user_embedding, self.book_embedding = self.create_embedding_layer()
        self.cf_layer = self.create_CF_layer()
        self._init_embedding_weights(self.user_embedding)


    def create_embedding_layer(self):
        """Creates the embedding layer"""
        user_in_shape = self.m + self.m_f
        book_in_shape = self.n
        out_shape = self.embedding_dim
        user_embedding = torch.nn.Embedding(num_embeddings=user_in_shape, embedding_dim=out_shape)
        book_embedding = torch.nn.Embedding(num_embeddings=book_in_shape, embedding_dim=out_shape)
        return user_embedding, book_embedding
    
    def _init_embedding_weights(self, embedding_layer):
        """Initializes the embedding layer weights with a uniform distribution."""
        embedding_layer.weight.data.uniform_(0, 1)
    
    def init_weights(self):
        """Initializes the weights of the model."""
        self._init_embedding_weights(self.user_embedding)
        self._init_embedding_weights(self.book_embedding)
        for layer in self.cf_layer:
            if isinstance(layer, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(layer.weight)
    
    def create_CF_layer(self):
        """Creates the collaborative filtering layers. Uses the number of neurons specified in `cf_layer_neurons`."""
        num_layers = len(self.cf_layer_neurons)
        activation = torch.nn.ReLU()
        layers = []
        for i in range(num_layers):
            if i == 0:
                layers.append(torch.nn.Linear(self.embedding_dim*2, self.cf_layer_neurons[i]))
            else:
                layers.append(torch.nn.Linear(self.cf_layer_neurons[i-1], self.cf_layer_neurons[i]))
            layers.append(activation)
        layers.append(torch.nn.Linear(self.cf_layer_neurons[-1], 1))
        if config.use_sigmoid:
            layers.append(torch.nn.Sigmoid())
        else:
            layers.append(activation)
        return torch.nn.Sequential(*layers)
    
    def forward(self, user_input, book_input):
        """Forward pass of the model.
        
        Parameters
        ----------
        user_input: torch.Tensor
            Tensor containing the user input.
        book_input: torch.Tensor
            Tensor containing the book input.
        """
        user_index = user_input.long()
        user_embedded = self.user_embedding(user_index)
        book_index = book_input.long()
        book_embedded = self.book_embedding(book_index)
        # Concatenate the user and book embeddings to form one vector.
        x = torch.cat([user_embedded, book_embedded], dim=-1)
        x = self.cf_layer(x)
        x = torch.squeeze(x)
        return x

We have our model. Next, we will define a loss function and an optimizer. Then we will train the model.

### Loss Function

We will use MSE loss. We will also add the l1 and l2 regularization terms. This way, we can experiment with different regularization parameters.

In [69]:
class MSE_L1L2Loss(torch.nn.Module):
    def __init__(self, model, config):
        """Initializes the loss function.

        Parameters
        ----------
        model: torch.nn.Module
            The model to use for the loss function.
        l1_weight: float
            Weight for the L1 regularization term.
        l2_weight: float
            Weight for the L2 regularization term.
        """
        super().__init__()
        self.model = model
        self.l1_weight = config.l1_weight
        self.l2_weight = config.l2_weight

    def forward(self, y_hat, y):
        """The forward pass of the loss function."""
        mse_loss = torch.nn.functional.mse_loss(y_hat, y)
        l2_regularization = torch.tensor(0.)
        l1_regularization = torch.tensor(0.)
        for param in self.model.parameters():
            l2_regularization += torch.norm(param, 2)
            l1_regularization += torch.norm(param, 1)
        l1_regularization *= self.l1_weight
        l2_regularization *= self.l2_weight
        loss = mse_loss + l1_regularization + l2_regularization
        return loss

### Optimizer and Other Parameters

We can use the Adam optimizer. We will also create a learning rate scheduler.

In [145]:
wandb.init(project='Book_Recommendation', name='MLP_Model_Sigmoid_Embedding_Dim_2', tags=['MLP', 'Recommendation', "Pytorch"])
# somewhat constants
config = wandb.config
config.log_interval = 20
config.m = m
config.n = n
config.m_f = m_f
config.n_f = n_f
config.batch_size = 256
config.optimizer_str = "adam"
config.learning_rate = 0.001
config.scheduler_str = "plateau"
config.negative_samples_ratio = 0.4

In [146]:
# Variables
# For Dataset
config.features = False
config.normalize = True
train_dataset = BookDataset(users, books, train_interactions, config=config)
test_dataset = BookDataset(users, books, test_interactions, config=config)
train_df = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
test_df = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, shuffle=True)

# For Model
config.embedding_dim = 64
config.cf_layer_neurons = [40, 20]
config.use_sigmoid = True
model = MLPModel(config=config)

# For loss function
config.l1_weight = 0
config.l2_weight = 0
loss_func = MSE_L1L2Loss(model, config)
if config.optimizer_str == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
elif config.optimizer_str == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate)

if config.scheduler_str == "plateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, verbose=True, min_lr=1e-7)
elif config.scheduler_str == "step":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)


### Training

Excellent! We have our model. Now, we will train it.

In [147]:
def train_step(user, item, rating, optimizer):
    model.train()
    optimizer.zero_grad()
    prediction = model(user, item)
    loss = loss_func(prediction, rating)
    loss.backward()
    optimizer.step()
    wandb.log({"train_loss": loss.item()})
    return loss.item(), prediction

def test_step(user, item, rating):
    model.eval()
    prediction = model(user, item)
    loss = loss_func(prediction, rating)
    wandb.log({"test_loss": loss.item()})
    sample_rating_pred = test_dataset.decode_rating(prediction[0].item())
    sample_rating_true = test_dataset.decode_rating(rating[0].item())
    wandb.log({"sample_rating_pred": sample_rating_pred, "sample_rating_true": sample_rating_true})
    return loss.item()

In [148]:
wandb.watch(model, log="all")
epochs = 5
train_losses = []
test_losses = []
for epoch in tqdm(range(epochs)):
    batch = 0
    train_loss = 0
    test_loss = 0
    for user, item, rating in train_df:
        batch += 1
        loss, rating = train_step(user, item, rating, optimizer)
        train_loss += loss
        print(f"Epoch: {epoch}, Batch: {batch}/{len(train_df)}, Loss: {loss}", end = "\r")
    train_loss /= len(train_df)
    train_losses.append(train_loss)
    for user, item, rating in test_df:
        loss = test_step(user, item, rating)
        test_loss += loss
    test_loss /= len(test_df)
    test_losses.append(test_loss)
    scheduler.step(test_loss)
    print(f"Epoch: {epoch}, Train loss: {train_loss}, Test loss: {test_loss}")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0, Train loss: 0.16070952381544967, Test loss: 0.15654409622273793
Epoch: 1, Train loss: 0.14977387470120013, Test loss: 0.1469346324845058
Epoch: 2, Train loss: 0.13990050616868466, Test loss: 0.14190465866065607
Epoch: 3, Train loss: 0.13239042975795948, Test loss: 0.1371296004551213
Epoch: 4, Train loss: 0.1269333143907834, Test loss: 0.1372857580824596


In [149]:
wandb.finish()

sample_rating_pred,▃▃▅▃▃▅▃▃▃▃▃▆▂▆▂▆▂▅▅▃▂▂▅▁▆▅▅▃▅▆▆▇▇▁▃█▁▆▇█
sample_rating_true,▁██▆▁▁▇▅▁▁▇▁▁▇▇▅▅▁▅▅▇▆▇█▁▇▆▁▁██▅▄▁▁▅▁▇▇█
test_loss,▇▇▆▇██▇▇▅▆▅▇▂▅▆▆▃▆▃▃▆▂▇▅▁▂▅▂▂▃▅▃▁▂▂▅▄▃▂▃
train_loss,█▇▇▇▆▆▆▆▅▆▇▇▅▅▆▄▄▂▅▃▅▄▂▃▅▄▃▃▃▄▂▂▄▂▂▂▁▂▁▂
sample_rating_pred,6.0
sample_rating_true,3.0
test_loss,0.1449
train_loss,0.13632


Using normalized data with sigmoid is giving the best result.


In [150]:
user_input, book_input, targets = next(iter(test_df))

In [151]:
pred = model(user_input, book_input)

In [152]:
targets

tensor([0.0000, 0.9000, 0.9000, 0.8000, 0.0000, 0.0000, 0.0000, 0.9000, 0.8000,
        0.0000, 0.0000, 0.7000, 0.6000, 0.5000, 0.8000, 0.0000, 0.0000, 0.8000,
        1.0000, 0.8000, 0.9000, 0.0000, 1.0000, 0.7000, 0.7000, 0.9000, 0.0000,
        0.0000, 0.3000, 1.0000, 0.0000, 1.0000, 0.0000, 0.7000, 1.0000, 0.0000,
        0.7000, 0.5000, 0.8000, 0.0000, 0.7000, 0.8000, 0.0000, 0.7000, 0.6000,
        0.0000, 0.0000, 1.0000, 0.8000, 1.0000, 0.0000, 0.8000, 0.8000, 0.9000,
        0.7000, 0.0000, 0.0000, 0.7000, 0.7000, 0.0000, 0.5000, 0.9000, 0.7000,
        0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.7000, 0.7000, 0.0000, 0.8000,
        0.7000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7000,
        0.9000, 0.7000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.7000, 0.8000,
        0.0000, 0.9000, 0.7000, 0.0000, 1.0000, 0.6000, 0.8000, 0.8000, 0.8000,
        0.9000, 0.9000, 0.7000, 0.9000, 0.7000, 0.6000, 0.0000, 0.0000, 1.0000,
        1.0000, 0.8000, 0.6000, 0.9000, 

In [153]:
pred

tensor([0.2631, 0.1657, 0.8339, 0.6654, 0.3361, 0.5269, 0.3439, 0.6599, 0.5768,
        0.1337, 0.3506, 0.4981, 0.6131, 0.6089, 0.5697, 0.3241, 0.4278, 0.8464,
        0.8134, 0.3116, 0.4886, 0.4040, 0.3292, 0.2744, 0.5992, 0.3515, 0.2701,
        0.3002, 0.7850, 0.2551, 0.4636, 0.5408, 0.4053, 0.3488, 0.4705, 0.3538,
        0.7446, 0.3910, 0.6504, 0.4574, 0.4509, 0.2598, 0.4134, 0.7760, 0.7726,
        0.2326, 0.6744, 0.5321, 0.4582, 0.6402, 0.2109, 0.3758, 0.5220, 0.3016,
        0.6751, 0.3863, 0.4690, 0.4374, 0.2419, 0.3067, 0.4495, 0.2553, 0.5197,
        0.1834, 0.4100, 0.3394, 0.5648, 0.5805, 0.5820, 0.1327, 0.3698, 0.8139,
        0.2320, 0.4330, 0.5554, 0.2710, 0.2383, 0.2298, 0.3110, 0.4650, 0.6527,
        0.7670, 0.4119, 0.5186, 0.3451, 0.3827, 0.3105, 0.3094, 0.7133, 0.3180,
        0.1511, 0.7202, 0.6250, 0.6934, 0.7238, 0.7627, 0.1353, 0.7255, 0.5959,
        0.4841, 0.7374, 0.5581, 0.4351, 0.3193, 0.6245, 0.4703, 0.3735, 0.6219,
        0.4314, 0.5662, 0.6748, 0.5413, 